In [1]:
import datetime
import logging
import os
import numpy as np
import matplotlib.pyplot as plt
import random
## Only when developing with VSCode in my repo 
# os.chdir('/home/nicolas/codeRepo/OceanPlatformControl') # here in Azure
# print(os.getcwd())
##
import pandas as pd
from ocean_navigation_simulator.problem_factories.Constructor import (
    Constructor,
)
from ocean_navigation_simulator.reinforcement_learning.missions.MissionGenerator import (
    MissionGenerator,
)
from ocean_navigation_simulator.reinforcement_learning.runners.GenerationRunner import (
    GenerationRunner,
)
from ocean_navigation_simulator.reinforcement_learning.missions.CachedNavigationProblem import (
    CachedNavigationProblem,
)
from ocean_navigation_simulator.utils.misc import set_arena_loggers
# These lines in VSCode to avoid reloading the kernel when changes are made to the external kernel
%load_ext autoreload
%autoreload 2
# Settings for where the problem csv is saved
my_path = os.getcwd()
results_folder = os.path.join(my_path, "tmp/missions/")
os.makedirs(results_folder, exist_ok=True)

## Generate Random Missions

In [2]:
# Settings for where the problem csv is saved
my_path = os.getcwd()
results_folder = os.path.join(my_path, "tmp/missions/")
os.makedirs(results_folder, exist_ok=True)
filename = 'problemsGOM_30_pltf_FC_HC.csv'

### 1. Arena Configuration and Mission configuration

In [3]:
arena_config = {
    "casadi_cache_dict": {"deg_around_x_t": 2.0, "time_around_x_t": 432000},
    "platform_dict": {
        "battery_cap_in_wh": 400.0,
        "u_max_in_mps": 0.1,
        "motor_efficiency": 1.0,
        "solar_panel_size": 0.5,
        "solar_efficiency": 0.2,
        "drag_factor": 675.0,
        "dt_in_s": 600.0,
    },
    "use_geographic_coordinate_system": True,
    "spatial_boundary": None,
    "ocean_dict": {
        "hindcast": {
            "field": "OceanCurrents",
            "source": "hindcast_files",
            "source_settings": {
                "folder": "data/miss_gen_hindcast/",
                "local": False,
                "source": "HYCOM",
                "type": "hindcast",
                "currents": "total",
                "region": "GOM"
            },
        },
        "forecast":{
            "field": "OceanCurrents",
            "source": "forecast_files",
            "source_settings": {
                "folder": "data/miss_gen_forecast/",
                "local": False,
                "source": "Copernicus",
                "type": "forecast",
                "currents": "total",
                "region": "GOM",
            },
        },
    },
    "multi_agent_constraints": {
        "unit": "km",
        "communication_thrsld": 9,
        "epsilon_margin": 1,  # when add edges based on hysteresis
        "collision_thrsld": 0.2,
    },
}

# change to this if basic setup works
set_arena_loggers(logging.DEBUG)
logging.getLogger("MissionGenerator").setLevel(logging.DEBUG)

config = {
    "scenario_config": arena_config,
    ##### Target Sampling #####
    # HYCOM HC: lon [-98.0,-76.4000244140625], lat[18.1200008392334,31.92000007629394]
    # Copernicus FC: lon: [-98.0, -76.416664], lat: [18.083334, 30.0]
    # Combined: [-98.0, -76.416664], [18.1200008392334, 30.0]
    "x_range": [-96, -78],
    "y_range": [20, 29],
    # "x_range": [-160, -105],
    # "y_range": [15, 40],
    "t_range": [  # ['2022-08-01T00:00:00+00:00', '2022-08-30T00:00:00+00:00'],
        # Copernicus FC: 2022-04 until today, HYCOM Hindcast: 2021-09 until today
        datetime.datetime(year=2022, month=4, day=1, tzinfo=datetime.timezone.utc),
        datetime.datetime(year=2022, month=12, day=30, tzinfo=datetime.timezone.utc),
    ],
    "problem_timeout_in_h": 24*6, #to be considered as feasible
    "target_distance_from_land": 1.2,
    "problem_target_radius": 0.25,
    ##### HJ Planner #####
    "hj_specific_settings": {
        # 'grid_res' has to be smaller than target_radius to prevent hj_solver errors
        "grid_res": 0.04,  # Note: this is in deg lat, lon (HYCOM Global is 0.083 and Mexico 0.04)
        "direction": "multi-time-reach-back",
        "n_time_vector": 199,  # Note that this is the number of time-intervals, the vector is +1 longer because of init_time
        "accuracy": "high",
        "artificial_dissipation_scheme": "local_local",
        "run_without_x_T": True,
        "progress_bar": False,
        "T_goal_in_seconds": 3600 * 24 * 7, #7 days here #
    },
    "hj_planner_box": 2.0,
    ##### Start Sampling #####
    "feasible_missions_per_target": 1,
    "random_missions_per_target": 8,
    "multi_agent": {
        "is_multi_agent": True,
        "nb_platforms": 30,
        "minimum_dist_km": 2,
        "sampling_range_radius_km": 60,
        "sample_range_ttr_h": [72, 132], # this has to be a larger than the "feasible_mission_time_in_h"
        # as the multi-agent points are sampled from "feasible_mission_time_in_h" with added noise which might 
        # increase their ttr by quite a lot (so either this interval has to be increased or the other reduced)
    },
    "min_distance_from_hj_frame": 1,
    "min_distance_from_land": 0.4,
    "feasible_mission_time_in_h": [96, 104],
    "random_min_distance_from_target": 0.5,
    ##### Actions #####
    "plot_batch": False,
    "animate_batch": False,
    "cache_forecast": False,
    "cache_hindcast": False,
}

### 2. Run generation

In [4]:
nb_workers = 1 # nb_of different targets
all_problems = []
for worker in range(nb_workers):
    mission_generator = MissionGenerator(
        config=config
        | {
            "seed": 2022,#int(random.uniform(0, 1000000)),
            "cache_folder": results_folder + str(worker) + "_",
        },
    )
    problems, _, _ = mission_generator.cache_batch()
    all_problems.extend(problems)
df = pd.DataFrame([problem.to_dict() for problem in all_problems])
df.to_csv(results_folder + filename)

Starting to connect to c3


INFO:c3:Utils: Connect to c3 (10.63s)
INFO:arena.factory:Downloading 8 files to 'data/miss_gen_hindcast/'.
INFO:arena.factory:File already downloaded: 'GOMu0.04-expt_90.1m000-2022-2022-07-20T00:00:00Z-2022-07-20T23:00:00Z.nc', 0.8MB.
INFO:arena.factory:File already downloaded: 'GOMu0.04-expt_90.1m000-2022-2022-07-21T00:00:00Z-2022-07-21T23:00:00Z.nc', 0.8MB.
INFO:arena.factory:File already downloaded: 'GOMu0.04-expt_90.1m000-2022-2022-07-22T00:00:00Z-2022-07-22T23:00:00Z.nc', 0.8MB.
INFO:arena.factory:File already downloaded: 'GOMu0.04-expt_90.1m000-2022-2022-07-23T00:00:00Z-2022-07-23T23:00:00Z.nc', 0.8MB.
INFO:arena.factory:File already downloaded: 'GOMu0.04-expt_90.1m000-2022-2022-07-24T00:00:00Z-2022-07-24T23:00:00Z.nc', 0.8MB.
INFO:arena.factory:File already downloaded: 'GOMu0.04-expt_90.1m000-2022-2022-07-25T00:00:00Z-2022-07-25T23:00:00Z.nc', 0.8MB.
INFO:arena.factory:File already downloaded: 'GOMu0.04-expt_90.1m000-2022-2022-07-26T00:00:00Z-2022-07-26T23:00:00Z.nc', 0.8MB.
INFO

In [5]:
all_problems[0].to_c3_mission_config()

{'x_0': [{'lon': -95.25455474853516,
   'lat': 25.441238403320312,
   'date_time': '2022-09-04T22:28:42+00:00'},
  {'lon': -95.28489685058594,
   'lat': 25.48339080810547,
   'date_time': '2022-09-04T22:28:42+00:00'},
  {'lon': -95.34382629394531,
   'lat': 25.50655746459961,
   'date_time': '2022-09-04T22:28:42+00:00'},
  {'lon': -95.34134674072266,
   'lat': 25.580486297607422,
   'date_time': '2022-09-04T22:28:42+00:00'},
  {'lon': -95.24158477783203,
   'lat': 25.377668380737305,
   'date_time': '2022-09-04T22:28:42+00:00'},
  {'lon': -95.36258697509766,
   'lat': 25.484975814819336,
   'date_time': '2022-09-04T22:28:42+00:00'},
  {'lon': -95.28181457519531,
   'lat': 25.366283416748047,
   'date_time': '2022-09-04T22:28:42+00:00'},
  {'lon': -95.41698455810547,
   'lat': 25.462541580200195,
   'date_time': '2022-09-04T22:28:42+00:00'},
  {'lon': -95.23809814453125,
   'lat': 25.37932014465332,
   'date_time': '2022-09-04T22:28:42+00:00'},
  {'lon': -95.25218963623047,
   'lat': 25

### 3. Visualize generated Navigation Problems

In [ ]:
GenerationRunner.plot_ttr_histogram(results_folder, pb_filename=filename)
# visualize only first target and associated starts (which defines a c3 "mission") 
GenerationRunner.plot_starts_and_targets(
    results_folder=results_folder, pb_filename=filename, scenario_config=arena_config,
)